# Random Walker Experiments

Author: K. Voudouris, 2023 (c) All Rights Reserved.

Contact: kv301@cam.ac.uk; k.voudouris14@googlemail.com; [Twitter @KozzyVoudouris](https://twitter.com/KozzyVoudouris); [GitHub @kozzy97](https://github.com/kozzy97)

Date: July 2023

This script runs a series of random walkers on the object permanence tests and stores the results in a MySQL database. It relies on a few things:

1. All the dependencies are installed, particularly that animalai is installed properly. I recommend using a conda environment and setting up an ipykernel for running this notebook.
2. AnimalAI is installed as an executable in the `env` folder.
3. A recent installation of MySQL, configured with a database, user, and password, as well as the local (or remote) address to store on. MySQL WorkBench is a good IDE for interacting with MySQL (this was created with WorkBench 8.0)
4. A CSV file in the same directory as this notebook called `databaseConnectionDetails.csv`, containing columns `database_name`, `hostname`, `username`, and `password` for database connection, with the values in the next row. This is gitignored.

In [2]:
import numpy as np
import os
import pandas as pd
import pymysql
import random
import sys

from animalai.envs.environment import AnimalAIEnvironment
from collections import deque
from gym_unity.envs import UnityToGymWrapper
from scipy.special import softmax

import sys
sys.path.append('../src')

from randomWalkers import RandomWalker #import the random walker class
from yamlHandling import find_yaml_files #this function finds the yaml files in a directory.
from yamlHandling import yaml_combinor #this function combines a batch of yaml files and saves the output in a temporary folder. This means we can run inference on batches of tests at once.

## Database Connection

A function for connecting to the database.

In [3]:
def databaseConnector(databaseCredentialsCSV: str):
    databaseCredentials = pd.read_csv(databaseCredentialsCSV)
    
    connection = pymysql.connect(
        host=databaseCredentials['hostname'].iloc[0],
        user=databaseCredentials['username'].iloc[0],
        password=databaseCredentials['password'].iloc[0],
        database=databaseCredentials['database_name'].iloc[0]
        )

    mycursor = connection.cursor()

    print(f"Connected to database: `{databaseCredentials['database_name'].iloc[0]}` at `{databaseCredentials['hostname'].iloc[0]}` with user `{databaseCredentials['username'].iloc[0]}`.")

    return mycursor, connection

In [4]:
mycursor, connection = databaseConnector('databaseConnectionDetails.csv')

mycursor.close()

print("Connection checked and closed.")

Connected to database: `opiaagets` at `localhost` with user `agenttester`.
Connection checked and closed.


## Paths

Provide the paths to the directory containing the configs being tested over, as well as the path the animal ai environment. Finally, provide a location for generating temporary files of combined configs. This defaults to the parent directory of the github repository, to prevent results being pushed accidentally.

In [5]:
configuration_folder = "../../configs/tests_agents"

env_path = "../../env/AnimalAI"

temp_folder_location = "../../.."

## Add All Tasks In Directory To Database

Iterate through the directory and find yaml files and their task names.

In [6]:
rerunInstanceTable = False

mycursor, connection = databaseConnector('databaseConnectionDetails.csv')

yaml_files, task_names = find_yaml_files(configuration_folder)

if rerunInstanceTable:
    dropTable = "DROP TABLE IF EXISTS randomwalkerinstanceresults, randomwalkerintrainstanceresults, instances;"
    mycursor.execute(dropTable)
    
    sql = "CREATE TABLE instances(instanceid INT AUTO_INCREMENT PRIMARY KEY, instancename VARCHAR(750) UNIQUE NOT NULL);"
    mycursor.execute(sql)

for instance in task_names:
    try:
        insertQuery = "INSERT INTO instances(instancename) VALUES('" + str(instance) + "');"
        mycursor.execute(insertQuery)
        connection.commit()
    except:
        print(f"Task {instance} has already been added to this table. Moving to next.")

mycursor.close()



Connected to database: `opiaagets` at `localhost` with user `agenttester`.
Task OP-STC-Allo-CVChick-1Occluder-Left-Beige-0-OPQ.yml has already been added to this table. Moving to next.
Task OP-STC-Allo-CVChick-1Occluder-Left-Brown-0-OPQ.yml has already been added to this table. Moving to next.
Task OP-STC-Allo-CVChick-1Occluder-Left-DarkBlue-0-OPQ.yml has already been added to this table. Moving to next.
Task OP-STC-Allo-CVChick-1Occluder-Left-DarkGreen-0-OPQ.yml has already been added to this table. Moving to next.
Task OP-STC-Allo-CVChick-1Occluder-Left-Grey-0-OPQ.yml has already been added to this table. Moving to next.
Task OP-STC-Allo-CVChick-1Occluder-Left-OffWhite-0-OPQ.yml has already been added to this table. Moving to next.
Task OP-STC-Allo-CVChick-1Occluder-Left-RND-0-OPQ.yml has already been added to this table. Moving to next.
Task OP-STC-Allo-CVChick-1Occluder-Right-Beige-0-OPQ.yml has already been added to this table. Moving to next.
Task OP-STC-Allo-CVChick-1Occluder-Ri

## Agents

Dictionaries of parameters to define some random walker agents. Note that these agents move quite slowly. They take one step forwards then pause for one step. This is to reduce the effect of momentum in the environment so their paths are more accurate.

The first agent is a simple fixed-step random walker. The walker moves 15 steps forwards, and then turns for 10 steps, with a probability of 0.5 of turning left or right. This serves as a very simple baseline.

In [7]:
fixed_action_walker = {'saccade_distribution' : 'fixed',
                       'angle_distribution' : 'fixed',
                       'max_saccade_length' : 15,
                       'max_angle_steps': 10,
                       'angle_fixed_randomise_turn': True,
                       'aai_seed' : 2023,
                       'agent_tag' : 'Random Walker Fixed Forwards Saccade 15 Angle 10'}

The next agent is a common model of foraging in animals. It is a Rayleight flight. Saccades are sampled from the normal distribution with a mean of 15 and a standard deviation of 5. If negative values are sampled the agent goes backwards that many steps, so the agent will tend to go forwards but occasionally go backwards. Angles are sampled uniformly between 1 and 60 steps of turning. Each turn is 6 degrees, making 60 steps be a full turn. Choice of left and right is randomised with probability 0.5.

In [8]:
rayleigh_flight = {'saccade_distribution' : 'normal',
                   'angle_distribution' : 'uniform',
                   'saccade_norm_mu' : 15,
                   'saccade_norm_sig' : 5,
                   'max_angle_steps': 60,
                   'angle_fixed_randomise_turn': True,
                   'backwards_action' : True,
                   'aai_seed' : 2023,
                   'agent_tag' : 'Random Walker Rayleigh Flight Norm Saccade mu 15 sig 5 Uniform Angles 1 to 60'}

The next agent is another common model of foraging in animals. It is a levy/cauchy flight. Saccades are sampled from the cauchy distribution, centred around 15. If negative values are sampled the agent goes backwards that many steps, so the agent will tend to go forwards but occasionally it will go backwards. Angles are sampled uniformly between 1 and 60 steps of turing. Each turn is 6 degrees, making 60 steps be a full turn. Choice of left and right is randomised with probability 0.5.

In [9]:
levy_flight = {'saccade_distribution' : 'cauchy',
               'angle_distribution' : 'uniform',
               'saccade_cauchy_mode' : 15,
               'max_angle_steps': 60,
               'angle_fixed_randomise_turn': True,
               'backwards_action' : True,
               'aai_seed' : 2023,
               'agent_tag' : 'Random Walker Levy Flight Cauchy Saccade mode 15 Uniform Angles 1 to 60'}

The next agent is an extension of the Rayleigh flight. Angles are also sampled from a normal distribution centred around 0 with a standard deviation of 4 so it tends to explore in front of it (standard deviation of 24 degrees to each side) but occasionally turns further around. 

In [10]:
rayleigh_flight_norm_angles = {'saccade_distribution' : 'normal',
                               'angle_distribution' : 'normal',
                               'saccade_norm_mu' : 15,
                               'saccade_norm_sig' : 5,
                               'angle_norm_mu': 0,
                               'angle_norm_sig': 24,
                               'backwards_action' : True,
                               'aai_seed' : 2023,
                               'agent_tag' : 'Random Walker Rayleigh Flight Norm Saccade mu 15 sig 5 Norm Angles mu 0 sd'}

The next agent is an extension of the Levy/Cauchy flight. Angles are also sampled from the Cauchy distribution centred around 0, so it tends to explore in front of it but given the Cauchy distribution might take big turns occasionally (due to heavy tails).

In [11]:
levy_flight_cauchy_angles = {'saccade_distribution' : 'cauchy',
                             'angle_distribution' : 'cauchy',
                             'saccade_cauchy_mode' : 15,
                             'angle_cauchy_mode': 0,
                             'angle_fixed_randomise_turn': True,
                             'backwards_action' : True,
                             'aai_seed' : 2023,
                             'agent_tag' : 'Random Walker Levy Flight Cauchy Saccade mode 15 Cauchy Angles mode 0'}

In [12]:
rerunAgentTable = True

mycursor, connection = databaseConnector('databaseConnectionDetails.csv')

if rerunAgentTable:
    dropTable = "DROP TABLE IF EXISTS randomwalkerinstanceresults, randomwalkerintrainstanceresults, randomwalkers;"
    mycursor.execute(dropTable)
    
    sql = "CREATE TABLE `randomwalkers` (`agentid` INT AUTO_INCREMENT PRIMARY KEY, `agent_tag` VARCHAR(300) UNIQUE, `aai_seed` INT, `max_saccade_length` INT,  `max_angle_steps` INT,  `saccade_distribution` VARCHAR(10), `angle_distribution` VARCHAR(10), `saccade_norm_mu` FLOAT(8), `saccade_norm_sig` FLOAT(8), `saccade_beta_alpha` FLOAT(8), `saccade_beta_beta` FLOAT(8), `saccade_cauchy_mode` FLOAT(8), `saccade_gamma_kappa` FLOAT(8), `saccade_gamma_theta` FLOAT(8), `saccade_weibull_alpha` FLOAT(8), `saccade_poisson_lambda` FLOAT(8), `angle_fixed_randomise_turn` BOOL, `angle_norm_mu` FLOAT(8), `angle_norm_sig` FLOAT(8), `angle_beta_alpha` FLOAT(8), `angle_beta_beta` FLOAT(8), `angle_cauchy_mode` FLOAT(8), `angle_gamma_kappa` FLOAT(8), `angle_gamma_theta` FLOAT(8), `angle_weibull_alpha` FLOAT(8), `angle_poisson_lambda` FLOAT(8), `angle_correlation` FLOAT(8), `backwards_action` BOOL);"
    mycursor.execute(sql)

mycursor.close()

Connected to database: `opiaagets` at `localhost` with user `agenttester`.


In [19]:
def agentToDB (cur, dictionary : dict):
    keys = ', '.join(dictionary.keys())
    values = ', '.join(
        f"'{value}'" if not isinstance(value, (int, float, bool)) else str(int(value))
        if isinstance(value, bool) else str(value)
        for value in dictionary.values()
        )

    query = f"INSERT INTO randomwalkers({keys}) VALUES ({values});"

    cur.execute(query)
    
    return query

In [20]:
mycursor, connection = databaseConnector('databaseConnectionDetails.csv')

agentToDB(mycursor, fixed_action_walker)

agentToDB(mycursor, rayleigh_flight)

agentToDB(mycursor, levy_flight)

agentToDB(mycursor, rayleigh_flight_norm_angles)

agentToDB(mycursor, levy_flight_cauchy_angles)

connection.commit()

mycursor.close()

Connected to database: `opiaagets` at `localhost` with user `agenttester`.


## Run Inference And Store

Need to iterate through the dictionaries and run inference.

In [41]:
mycursor, connection = databaseConnector('databaseConnectionDetails.csv')

rerunInstanceResults = True

if rerunInstanceResults:
    dropInstanceResultsTables = "DROP TABLE IF EXISTS randomwalkerinstanceresults, randomwalkerintrainstanceresults;"
    mycursor.execute(dropInstanceResultsTables)
    
    createInstanceTable = "CREATE TABLE randomwalkerinstanceresults(instanceid INT NOT NULL, agentid INT NOT NULL, finalreward FLOAT(53), FOREIGN KEY (instanceid) REFERENCES instances(instanceid), FOREIGN KEY(agentid) REFERENCES randomwalkers(agentid), PRIMARY KEY (instanceid, agentid));"
    mycursor.execute(createInstanceTable)

    createIntraInstanceTable = "CREATE TABLE randomwalkerintrainstanceresults(instanceid INT NOT NULL, agentid INT NOT NULL, step INT NOT NULL, actiontaken INT NOT NULL, stepreward FLOAT(53), xvelocity FLOAT(32), yvelocity FLOAT(32), zvelocity FLOAT(32), xpos FLOAT(32), ypos FLOAT(32), zpos FLOAT(32), FOREIGN KEY (instanceid) REFERENCES instances(instanceid), FOREIGN KEY(agentid) REFERENCES randomwalkers(agentid), PRIMARY KEY(instanceid, agentid, step));"
    mycursor.execute(createIntraInstanceTable)

mycursor.close()


Connected to database: `opiaagets` at `localhost` with user `agenttester`.


In [42]:
batch_size = 40

port = 5000 + random.randint( #create random base port.
    0, 2000
)

mycursor, connection = databaseConnector('databaseConnectionDetails.csv')

for i in range(0, len(yaml_files), batch_size):
    if ((i + batch_size)-1) > len(yaml_files) or batch_size > len(yaml_files):
        upper_bound = len(yaml_files)
    else:
        upper_bound = ((i + batch_size)-1)

    print(f"Running inferences on {upper_bound} files.")
    
    batch_files = yaml_files[i:upper_bound]

    batch_file_names = task_names[i:upper_bound]

    batch_temp_file_name = "TempConfig_" + str(i) + ".yml"

    config_file_path = yaml_combinor(file_list = batch_files, temp_file_location=temp_folder_location, stored_file_name = batch_temp_file_name)

    print("Opening AAI Environment")

    temp_port = port + i # increment through ports to prevent calling the same socket.

    aai_env = AnimalAIEnvironment( 
        inference=agent_inference, #Set true when watching the agent
        seed = aai_seed,
        worker_id=aai_seed,
        file_name=env_path,
        arenas_configurations=config_file_path,
        base_port=temp_port,
        useCamera=False,
        resolution=36,
        useRayCasts=False,
        )
    
    env = UnityToGymWrapper(aai_env, uint8_visual=False, allow_multiple_obs=True, flatten_branched=True)

    obs = env.reset()  

    behavior = list(aai_env.behavior_specs.keys())[0] # by default should be AnimalAI?team=0


    for _instance in range(len(batch_files)): 

        random.seed(aai_seed)

        agent = RandomWalker(max_step_length = max_step_length, 
                             step_length_distribution = step_length_distribution, 
                             norm_mu = norm_mu, 
                             norm_sig = norm_sig, 
                             beta_alpha = beta_alpha, 
                             beta_beta = beta_beta, 
                             cauchy_mode = cauchy_mode, 
                             gamma_kappa = gamma_kappa, 
                             gamma_theta = gamma_theta, 
                             weibull_alpha = weibull_alpha, 
                             poisson_lambda = poisson_lambda, 
                             action_biases = action_biases, 
                             prev_step_bias = prev_step_bias, 
                             remove_prev_step = remove_prev_step)
        
        done = False
        
        initialActionAgent = agent
        initialActionAgent.prev_step_bias = 0 #select a random action according to the biases. There is no previous step bias as there is no previous step at the start of an episode!

        previous_action = initialActionAgent.get_new_action(prev_step=0)

        getInstanceIDQuery = "SELECT instanceid FROM instances WHERE instancename = '" + task_names[_instance] + "';"

        mycursor.execute(getInstanceIDQuery)
        
        instanceid = int(mycursor.fetchone()[0]) #fetch the result, there should just be 1.

        episodeReward = 0 # reward starts at 0 for each episode.
        
        step_counter = 0
        
        while not done:

            step_list = agent.get_num_steps(prev_step = previous_action)
        
            for action in step_list:
            
                obs, reward, done, info = env.step(int(action))

                dec, term = aai_env.get_steps(behavior)
                env.render()
                previous_action = action

                try:
                    intraInstanceQuery = "INSERT INTO randomwalkerintrainstanceresults(instanceid, agentid, step, actiontaken, stepreward, xvelocity, yvelocity, zvelocity, xpos, ypos, zpos) VALUES (" + str(instanceid) + ", " + str(agentID) + ", " + str(step_counter) + ", " + str(action) + ", " + str(episodeReward) + ", " + str(obs[0][1]) + ", " + str(obs[0][2]) + ", " + str(obs[0][3]) + ", " + str(obs[0][4]) + ", " + str(obs[0][5]) + ", " + str(obs[0][6]) + ");"
                    mycursor.execute(intraInstanceQuery)
                    connection.commit()
            
                except:
                    print(f"There's something wrong with this step. Here's the query {intraInstanceQuery}")
                    pass
                
                step_counter += 1

                if len(term) > 0: #Episode is over
                    episodeReward += reward
                    print(F"Episode Reward: {episodeReward}")
                    
                    done = True
                    obs=env.reset()
                    break
                else:
                    episodeReward += reward

            if done:
                break # must exit the while loop if done in previous for loop (as break in if statement only breaks out of for loop)

            ## get new action for one step before repeating while loop.

            action = agent.get_new_action(prev_step = previous_action)
        
            obs, reward, done, info = env.step(int(action))

            try:
                intraInstanceQuery = "INSERT INTO randomwalkerintrainstanceresults(instanceid, agentid, step, stepreward, xvelocity, yvelocity, zvelocity, xpos, ypos, zpos) VALUES (" + str(instanceid) + ", " + str(agentID) + ", " + str(step_counter) + ", " + str(action) + ", " + str(episodeReward) + ", " + str(obs[0][1]) + ", " + str(obs[0][2]) + ", " + str(obs[0][3]) + ", " + str(obs[0][4]) + ", " + str(obs[0][5]) + ", " + str(obs[0][6]) + ");"
                mycursor.execute(intraInstanceQuery)
                connection.commit()
                
            except:
                pass
            step_counter += 1

            dec, term = aai_env.get_steps(behavior)
            env.render()
            previous_action = action
            if len(term) > 0: #Episode is over
                episodeReward += reward
                print(F"Episode Reward: {episodeReward}")
                    
                #episode_list.append(_instance)
                #reward_list.append(episodeReward)
                done = True
                obs=env.reset()
                break
            else:
                episodeReward += reward

        try:
            insertInstanceResults = "INSERT INTO randomwalkerinstanceresults(instanceid, agentid, finalreward) VALUES (" + str(instanceid) + ", " + str(agentID) + ", " + str(float(episodeReward)) + ");"
            mycursor.execute(insertInstanceResults)
            connection.commit()
        except:
            print("It looks like this agent has already been tested on this instance.")

    env.close()
    os.remove(config_file_path)

mycursor.close()

Connected to database: `opiaagets` at `localhost` with user `agenttester`.
Running inferences on 29 files.
Yaml files combined. Saved to C:/Users/kv301/Documents\TempConfig_0.yml
Opening AAI Environment
[INFO] Connected to Unity environment with package version 2.1.0-exp.1 and communication version 1.5.0
[INFO] Connected new brain: AnimalAI?team=0


c:\Users\kv301\anaconda3\envs\animalaiv3\lib\site-packages\gym\spaces\box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


Episode Reward: -0.9999999504943844
Episode Reward: -1.2755999471410178
Episode Reward: -0.9999999504943844
Episode Reward: -0.9999999505234882
Episode Reward: -0.9991999505436979
Episode Reward: -0.9991999505436979
Episode Reward: -1.1239999410463497
Episode Reward: -0.9995999505044892
Episode Reward: -0.9995999505044892
Episode Reward: -1.9439999668393284
Episode Reward: -0.9995999505044892
Episode Reward: -0.9991999505436979
Episode Reward: -0.9991999505436979
Episode Reward: -0.9991999505436979
Episode Reward: -0.9999999505234882
Episode Reward: -0.9999999505234882
Episode Reward: -1.211600003123749
Episode Reward: -0.9999999505234882
Episode Reward: -0.9991999505436979
Episode Reward: -0.9991999505436979
Episode Reward: -0.9991999505436979
Episode Reward: -1.2024000035598874
Episode Reward: -0.9995999505044892
Episode Reward: -0.9995999505044892
Episode Reward: -0.9995999505044892
Episode Reward: -0.9995999505044892
Episode Reward: -0.9991999505436979
Episode Reward: -0.9991999505